In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.c

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 199 kB 52.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=52f9b4a29f382115ed0136edec0dd0ca2ec098c38019838eabf0b6c75a3934ee
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [4]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-06 20:43:46--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-07-06 20:43:47 (6.54 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [7]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Furniture_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-ChallengeD2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [8]:
df

DataFrame[marketplace: string, customer_id: int, review_id: string, product_id: string, product_parent: int, product_title: string, product_category: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]

In [9]:
totalvotes_df = df.filter("total_votes>=20")
totalvotes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41681546| RL8D0KJ0J9L0O|B00BWC1X3S|     328960153|Zinus 14 Inch Eli...|       Furniture|          5|          152|        165|   N|                Y|A solid VICTORY f...|I've been looking...| 2015-08-31|
|         US|   16806846|R1BEINAIQFBRJC|B007I81A60|      68465765|8" Night Therapy ...|       Furniture|          5|    

In [10]:
helpful_50_df = totalvotes_df.filter("helpful_votes/total_votes>=.50")
helpful_50_df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41681546| RL8D0KJ0J9L0O|B00BWC1X3S|     328960153|Zinus 14 Inch Eli...|       Furniture|          5|          152|        165|   N|                Y|A solid VICTORY f...|I've been looking...| 2015-08-31|
|         US|   16806846|R1BEINAIQFBRJC|B007I81A60|      68465765|8" Night Therapy ...|       Furniture|          5|    

In [11]:
print(helpful_50_df.count())

18155


In [12]:
vine_review_df = helpful_50_df.filter(helpful_50_df["vine"] == "Y")
vine_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   35119071|R2BQOD1R0228FN|B00H2RSA88|     405483618|Sleep Innovations...|       Furniture|          3|           17|         26|   Y|                N|An okay product. ...|Three-stars is co...| 2015-08-27|
|         US|   44737123| RC31RUPFOHBHQ|B0125QZ50G|     350975212|Zinus Viscolatex ...|       Furniture|          4|    

In [13]:
novine_review_df = helpful_50_df.filter(helpful_50_df["vine"] == "N")
novine_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41681546| RL8D0KJ0J9L0O|B00BWC1X3S|     328960153|Zinus 14 Inch Eli...|       Furniture|          5|          152|        165|   N|                Y|A solid VICTORY f...|I've been looking...| 2015-08-31|
|         US|   16806846|R1BEINAIQFBRJC|B007I81A60|      68465765|8" Night Therapy ...|       Furniture|          5|    

In [14]:
from pyspark.sql.functions import count
# Total number of reviews for each dataframe from Step 3 and Step 4:
vine_review_count = vine_review_df.count()
print("Total Number of Reviews PAID and 'helpful': %f" % vine_review_count)

novine_review_count = novine_review_df.count()
print("Total Number of Reviews NON-PAID and 'helpful': %f" % novine_review_count)

Total Number of Reviews PAID and 'helpful': 136.000000
Total Number of Reviews NON-PAID and 'helpful': 18019.000000


In [15]:
star5_vine_df = vine_review_df.filter(novine_review_df["star_rating"]=="5")
print("Total Number of 5-star Reviews PAID and 'helpful': %f" % star5_vine_df.count())


star5_novine_df = novine_review_df.filter(novine_review_df["star_rating"]=="5")
print("Total Number of 5-star Reviews NON-PAID and 'helpful': %f" % star5_novine_df.count())

Total Number of 5-star Reviews PAID and 'helpful': 74.000000
Total Number of 5-star Reviews NON-PAID and 'helpful': 8482.000000


In [ ]:
star5_vine_pct = (star5_vine_df.count()/vine_review_count)
print("Percent of 5-Star Furniture Reviews from PAID, 'helpful' dataset: %f" % star5_vine_pct)

star5_novine_pct = (star5_novine_df.count()/novine_review_count)
print("Percent of 5-Star Furniture Reviews from NON-PAID, 'helpful' dataset: %f" % star5_novine_pct)

In [16]:
total_reviews_count = df.count()
print("Total Number of Furniture Reviews: %f" % total_reviews_count)

help_reviews_count = helpful_50_df.count()
print("Total Number of 'Helpful' Furniture Reviews: %f" % help_reviews_count)

Total Number of Furniture Reviews: 792113.000000
Total Number of 'Helpful' Furniture Reviews: 18155.000000


In [17]:
star5_df = df.filter(df["star_rating"] == '5')
star5_df.show()

star5_count = star5_df.count()
print("Total Number of 5-Star Furniture Reviews: %f" % star5_count)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|            0|          0|   N|                Y|          Five Stars|          Great item| 2015-08-31|
|         US|    1272331|R1AIMEEPYHMOE4|B0030MPBZ4|     124663823|Bathroom Vanity T...|       Furniture|          5|    

In [18]:
star5_help_df = helpful_50_df.filter(helpful_50_df["star_rating"] == '5')
star5_help_df.show()

star5_help_count = star5_help_df.count()
print("Total Number of 5-Star 'Helpful' Furniture Reviews: %f" % star5_help_count)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41681546| RL8D0KJ0J9L0O|B00BWC1X3S|     328960153|Zinus 14 Inch Eli...|       Furniture|          5|          152|        165|   N|                Y|A solid VICTORY f...|I've been looking...| 2015-08-31|
|         US|   16806846|R1BEINAIQFBRJC|B007I81A60|      68465765|8" Night Therapy ...|       Furniture|          5|    

In [19]:
vine_star5_all_df = star5_df.filter(star5_df["vine"] == "Y")
vine_star5_all_count = vine_star5_all_df.count()

vine_star5_all_pct = (vine_star5_all_count/star5_count)
print("Number of 5-Star Furniture Reviews PAID: %f" % vine_star5_all_count)
print("Percent of 5-Star Furniture Reviews PAID: %f" % vine_star5_all_pct)
 
#5-star Non-Paid reviews
novine_star5_all_df = star5_df.filter(star5_df["vine"] == "N")
novine_star5_all_count = novine_star5_all_df.count()

novine_star5_all_pct = novine_star5_all_count/star5_count
print("Number of 5-Star Furniture Reviews NON-PAID: %f" % novine_star5_all_count)
print("Percent of 5-Star Furniture Reviews NON-PAID: %f" % novine_star5_all_pct)

Number of 5-Star Furniture Reviews PAID: 1356.000000
Percent of 5-Star Furniture Reviews PAID: 0.003029
Number of 5-Star Furniture Reviews NON-PAID: 446360.000000
Percent of 5-Star Furniture Reviews NON-PAID: 0.996971


In [ ]:
vine_star5_help_df = star5_help_df.filter(star5_help_df["vine"] == "Y")
vine_star5_help_count = vine_star5_help_df.count()

vine_star5_help_pct = (vine_star5_help_count/star5_help_count)
print("Number of 5-Star, 'Helpful' Furniture Reviews PAID: %f" % vine_star5_help_count)
print("Percent of 5-Star,'Helpful'Furniture Reviews PAID: %f" % vine_star5_help_pct)
 
#5-star Non-Paid "helpful" reviews
novine_star5_help_df = star5_help_df.filter(star5_help_df["vine"] == "N")
novine_star5_help_count = novine_star5_help_df.count()

novine_star5_help_pct = novine_star5_help_count/star5_help_count
print("Number of 5-Star, 'Helpful' Furniture Reviews NON-PAID: %f" % novine_star5_help_count)
print("Percent of 5-Star,'Helpful'Furniture Reviews NON-PAID: %f" % novine_star5_help_pct)